In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import inv
import math as m
from math import sqrt
import sympy as sp
from sympy import collect, simplify, expand, fraction, latex
from sympy.integrals import laplace_transform as laplace
from sympy.integrals import inverse_laplace_transform as ilaplace
from IPython.display import display, Markdown, Math
import control as co
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
sp.init_printing(use_latex='mathjax')
plt.rcParams['figure.figsize'] = [20, 10]

In [2]:
class numden_coeff:
    def __init__(self, expr, symb):
        self.num, self.denum = fraction(expr)
        self.symb = symb
        self.common_factor = None
        self.lst_denum_coeff = self.build_lst(self.denum)
        self.lst_num_coeff = self.build_lst(self.num)
        
    def build_lst(self, poly):
        order = sp.Poly(poly, self.symb).degree()
        lst = [expand(poly).coeff(self.symb**i) for i in range((order), 0, -1)]
        lst.append(poly.subs(self.symb,0))
        if (self.common_factor == None):
            self.common_factor = lst[0]
            
        lst = [simplify(lst[i]/self.common_factor) for i in range(order + 1)]
        return lst
    
    def disp(self):
        display(Markdown(r"Numerator coefficients (\beta)"), self.lst_num_coeff)
        display(Markdown(r"Denominator coefficients (alpha)"), self.lst_denum_coeff)

In [3]:
def tf_to_symbolic_fraction(tf):
    x = symbols('x')
    num, den = tfdata(tf)
    num = num[0][0]
    den = den[0][0]
    counter = 1
    length_num = len(num)
    length_den = len(den)

    sym_num, sym_den = 0,0

    if(length_num == 0):
        raise ValueError ("The fraction num should not be empty")
    elif(length_den == 0 ):
        raise ValueError("The fraction den should not be empty")

    for i in range(length_num):
        sym_num+=num[i]*(x**(length_num - counter))
        counter += counter
    counter = 1
    for i in range(length_den):
        sym_den+=den[i]*(x**(length_den - counter))
        counter += counter
    return sym_num/sym_den

In [4]:
a1, a2, b0, b1 = sp.symbols('a_{1} a_{2} b_{0} b_{1}')
am1, am2, bm0, bm1 = sp.symbols('a_{m1} a_{m2} b_{m0} b_{m1}')
T, t = sp.symbols("T t", positive=True, real=True)
n, b, zeta, omega = sp.symbols('n b zeta omega', real=True)
s, z, q = sp.symbols("s z q")


B = b # in s-domain
A = (s**2 + s*b) # in s-domain
G = B/A
G_pf = sp.apart(G/s, s)

temp = ilaplace(G_pf, s, t)
temp_nT = temp.subs(t, T*n)

display(Math("G(s) ="+latex(G)))
display(Math("\\frac{G(s)}{s} = G^{'}(s)="+latex(G_pf)))
display(Math("G^{'}(t) ="+latex(temp)))
display(Math("G^{'}(nT)  ="+latex(temp_nT)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [5]:
temp_z = T*(z**-1)/(1 - z**-1)**2 - 1/((1 - z**-1)*b) + b**-1/(1 - sp.exp(T*b)*z**-1)
temp_z

display(Math("G^{'}(z)  ="+latex(temp_z)))

<IPython.core.display.Math object>

In [6]:
H_z = collect(simplify(expand((1 - z**-1)*temp_z)), z)

display(Math("H_z =\;"+latex(H_z)))

<IPython.core.display.Math object>

$B = B^-=\;$ {{B}}

$B^+=1$

$$Deg(B^+) = 0$$
$$Deg(B^-) = 1$$

$$Deg(A) = Deg(A_m) = 2$$

$$Deg(S) = Deg(R) = Deg(A) - 1 = 2 - 1 = 1$$

$$Deg(R^{'}) = Deg(R) - Deg(B^+) = 1 - 0 = 1$$

$$Deg(A_0) = Deg(A) + Deg(R^{'}) - Deg(A_m) = 2 + 1 - 2 = 1$$

In [7]:
Bm_b = sp.Symbol("\\bar{B}_m")

A = q**2 + a1*q + a2 # in z-domain
B = b0*q + b1 # in z-domain
B_minus = B
B_plus = 1
H = B/A


Am = q**2 + am1*q + am2

#Final value theorem was used to find B_m_bar ==> Gm = Bm/Am = (B^-)*Bm_bar/Am = 1
Bm_bar = sp.solve(sp.Eq(B_minus*Bm_b/Am, 1),Bm_b)[0].subs(q,1)
Bm = B_minus*Bm_bar

# display(Math("\\bar{B}_m ="+latex(Bm_bar)))
# display(Math("B_m =\\bar{B}_mB^-="+latex(Bm)))

To achive unity gain, the final value theorem can implemented on $A_m$ to obtain the value of $B_m$. This is achived by the equation $B_m = A_m(1)q^m$.This way, when k goes to infinity ($q \rightarrow 1$), $G_m = 1$ and $Deg(B_m) = Deg(B)$. Therfore, if

$A_m =\;${{Am}}

then

$B_m =\;${{Bm}}

In [8]:
Hm = collect(simplify(expand(Bm/Am)), q)
display(Math("H_m ="+latex(Bm/Am)+"="+latex(Hm)))

<IPython.core.display.Math object>

In [9]:
r0, r1, s0, s1, a0 = sp.symbols('r_0 r_1 s_0 s_1 a_0')

A0 = q # might need to select something differnt if too sensitive to noise
S = s0*q + s1
R = r0*q+ r1
R_prime = R
T = Bm*A0/B_minus

The control polynomials become

$A_0 =\;${{A0}} (if system is too sensitive to noise, then move this pole farther from imaginary axis)

$S =\;${{S}}

$R =R^{'}=\;${{R}}

$T = A_0\frac{B_m}{B^{-}} =\;${{T}} 

In [10]:
LHS_coeffs = sp.Poly(A*R_prime + B_minus*S, q).coeffs()[::-1]
RHS_coeffs = sp.Poly(expand(A0*Am), q).coeffs()[::-1]

eq_3 = sp.Eq(LHS_coeffs[3], RHS_coeffs[2])
eq_2 = sp.Eq(LHS_coeffs[2], RHS_coeffs[1])
eq_1 = sp.Eq(LHS_coeffs[1], RHS_coeffs[0])
eq_0 = sp.Eq(LHS_coeffs[0], 0)

r_0 = sp.solve(eq_3, r0)[0]

# sub value for r0 into remaining equations
eq_2 = eq_2.subs(r0, r_0)
eq_1 = eq_1.subs(r0, r_0)
eq_0 = eq_0.subs(r0, r_0)

display(eq_2)
display(eq_1)
display(eq_0)

# Put equations into matrix form and solve for r1, s0 and s1
mat_lhs = sp.Matrix([[1, b0, 0], [a1, b1, b0], [a2, 0, b1]])
mat_rhs = sp.Matrix([[am1 - a1], [am2 - a2], [0]])

res = mat_lhs.inv()@mat_rhs
r_1 = res[0]
s_0 = res[1]
s_1 = res[2]

display(Math(latex(mat_lhs)+latex(sp.Eq(sp.Matrix([[r1], [s0], [s1]]), mat_rhs))))
display(Math(latex(sp.Eq(sp.Matrix([[r1], [s0], [s1]]), res))))

a_{1} + b_{0}⋅s₀ + r₁ = a_{m1}

a_{1}⋅r₁ + a_{2} + b_{0}⋅s₁ + b_{1}⋅s₀ = a_{m2}

a_{2}⋅r₁ + b_{1}⋅s₁ = 0

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [11]:
R_ = simplify(R.subs(r1, r_1))
S_ = simplify(S.subs([(s0, s_0), (s1, s_1)]))
T_ = simplify(T)

display(Math("R =\;"+latex(R_)))
display(Math("S =\;"+latex(S_)))
display(Math("T =\;"+latex(T_)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

We know that the pulse transfer function is of the form

$H(q) = \;${{H}} 

Therefore, the measurment model can be found to be

$y(t+2) = -a_1y(t+1) - a_2y(t) + b_0u(t+1) + b_1u(t)$

$\Rightarrow y(t) = -a_1y(t-1) - a_2y(t-2) + b_0u(t-1) + b_1u(t-2)$

Therfore

$y(t) = \phi^T(t)\theta = [-y(t-1)\; -y(t-2) \;\;u(t-1) \;\;u(t-2)][a_1 \;a_2 \; b_0 \; b_1]^T$

In [12]:
TR = simplify(T_/R_)
SR = simplify(S_/R_)

obj_TR = numden_coeff(TR, q)
obj_SR = numden_coeff(SR, q)

aTR = obj_TR.lst_denum_coeff
bTR = obj_TR.lst_num_coeff

aSR = obj_SR.lst_denum_coeff
bSR = obj_SR.lst_num_coeff

For $\frac{T}{R}$, the coefficients of the numerator and denominator are

$\alpha\frac{T}{R} =\;$ {{aTR}} 

and

$\beta\frac{T}{R} =\;$ {{bTR}}

while the coefficients of the numerator and denominator for $\frac{S}{R}$ are

$\alpha\frac{S}{R} =\;$ {{aSR}}

and

$\beta\frac{S}{R} =\;$ {{bSR}}

In [13]:
u_k_1, uc_k, y_k, y_k_1, y_k_2 = sp.symbols('u(k-1) u_{c}(k) y(k) y(k-1) y(k-2)')

uk = -u_k_1*aTR[1] + uc_k*bTR[0] - y_k*bSR[0] - y_k_1*bSR[1]

The control equation in terms of the plant and model parameters is

$u(t) = \;${{uk}}

In [40]:
zeta_val = 1
omega_val = 1
T_val = 1

# Create z-domain tf for reference model with true values for zeta, omega and T. 
# Results will be used to determin true values for am1 and am2
Bz, Az = co.tfdata(co.sample_system(co.tf([omega_val**2], [1, 2*zeta_val*omega_val, omega_val**2]),
                          Ts=T_val, 
                          method='zoh'))
am1_val = Az[0][0][1]
am2_val = Az[0][0][2]

# make control equation in terms of a1, a2, b0 and b1 and sub in true vals of am1 and am2
uk_subd = uk.subs([(r0, r_0), (r1, r_1), (s0, s_0), (s1, s_1), (am1, am1_val), (am2, am2_val)])
uk_func = sp.lambdify([u_k_1, uc_k, y_k, y_k_1, a1, a2, b0, b1], uk_subd) # turn eq into function

display(Math("a_{m1}="+latex(am1_val)))
display(Math("a_{m2}="+latex(am2_val)))
display(Math("u_k="+latex(uk_subd)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>